In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import scipy
from scipy import stats
# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

In [2]:
#Plotting
import plotly
import plotly.graph_objs as go
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
import plotly.express as px
import plotly.figure_factory as ff
from plotly import subplots
# Display plot in the same cell for plotly
init_notebook_mode(connected=True)

In [3]:
import sklearn
from sklearn import linear_model,metrics
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import r2_score, mean_squared_error, make_scorer

In [4]:
print('Numpy : Version ',np.__version__)
print('Pandas : Version ',pd.__version__)
print('Plotly : Version ',plotly.__version__)
print('Plotly Express : Version ',plotly.express.__version__)
print('Scikit-Learn : Version ',sklearn.__version__)

Numpy : Version  1.16.4
Pandas : Version  0.24.2
Plotly : Version  4.0.0
Plotly Express : Version  0.3.0
Scikit-Learn : Version  0.21.2


In [5]:
# Colors from material design to make visualizations look awesome!
material_blue = '#81d4fa'
dark_blue = '#1e88e5'
material_green = '#a5d6a7'
dark_green = '#43a047'
material_indigo = '#3f51b5'
material_red = '#f44336'
bg_color = '#212121'

In [6]:
# Importing the train dataset
df_train = pd.read_csv('train.csv')
df_train.head()

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,...,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
0,1,60,RL,65.0,8450,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,2,2008,WD,Normal,208500
1,2,20,RL,80.0,9600,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,5,2007,WD,Normal,181500
2,3,60,RL,68.0,11250,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,9,2008,WD,Normal,223500
3,4,70,RL,60.0,9550,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,2,2006,WD,Abnorml,140000
4,5,60,RL,84.0,14260,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,12,2008,WD,Normal,250000


In [7]:
# Importing the test dataset
df_test = pd.read_csv('test.csv')
df_test.head()

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,...,ScreenPorch,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition
0,1461,20,RH,80.0,11622,Pave,NaN,Reg,Lvl,AllPub,...,120,0,NaN,MnPrv,NaN,0,6,2010,WD,Normal
1,1462,20,RL,81.0,14267,Pave,NaN,IR1,Lvl,AllPub,...,0,0,NaN,NaN,Gar2,12500,6,2010,WD,Normal
2,1463,60,RL,74.0,13830,Pave,NaN,IR1,Lvl,AllPub,...,0,0,NaN,MnPrv,NaN,0,3,2010,WD,Normal
3,1464,60,RL,78.0,9978,Pave,NaN,IR1,Lvl,AllPub,...,0,0,NaN,NaN,NaN,0,6,2010,WD,Normal
4,1465,120,RL,43.0,5005,Pave,NaN,IR1,HLS,AllPub,...,144,0,NaN,NaN,NaN,0,1,2010,WD,Normal


In [8]:
# Column names
df_train.columns

Index(['Id', 'MSSubClass', 'MSZoning', 'LotFrontage', 'LotArea', 'Street',
       'Alley', 'LotShape', 'LandContour', 'Utilities', 'LotConfig',
       'LandSlope', 'Neighborhood', 'Condition1', 'Condition2', 'BldgType',
       'HouseStyle', 'OverallQual', 'OverallCond', 'YearBuilt', 'YearRemodAdd',
       'RoofStyle', 'RoofMatl', 'Exterior1st', 'Exterior2nd', 'MasVnrType',
       'MasVnrArea', 'ExterQual', 'ExterCond', 'Foundation', 'BsmtQual',
       'BsmtCond', 'BsmtExposure', 'BsmtFinType1', 'BsmtFinSF1',
       'BsmtFinType2', 'BsmtFinSF2', 'BsmtUnfSF', 'TotalBsmtSF', 'Heating',
       'HeatingQC', 'CentralAir', 'Electrical', '1stFlrSF', '2ndFlrSF',
       'LowQualFinSF', 'GrLivArea', 'BsmtFullBath', 'BsmtHalfBath', 'FullBath',
       'HalfBath', 'BedroomAbvGr', 'KitchenAbvGr', 'KitchenQual',
       'TotRmsAbvGrd', 'Functional', 'Fireplaces', 'FireplaceQu', 'GarageType',
       'GarageYrBlt', 'GarageFinish', 'GarageCars', 'GarageArea', 'GarageQual',
       'GarageCond', 'PavedDrive

In [9]:
# Shape => Tuple of no. of rows and columns
df_train.shape

(1460, 81)

### Observations:
**a) Dataset contains null values in some rows.**

**b) The categorical variables are in object type. Hence, they should be transformed to 'category' type in Pandas.**

In [10]:
df_train.describe()

,Id,MSSubClass,LotFrontage,LotArea,OverallQual,OverallCond,YearBuilt,YearRemodAdd,MasVnrArea,BsmtFinSF1,...,WoodDeckSF,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,MiscVal,MoSold,YrSold,SalePrice
count,1460.000000,1460.000000,1201.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1452.000000,1460.000000,...,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000
mean,730.500000,56.897260,70.049958,10516.828082,6.099315,5.575342,1971.267808,1984.865753,103.685262,443.639726,...,94.244521,46.660274,21.954110,3.409589,15.060959,2.758904,43.489041,6.321918,2007.815753,180921.195890
std,421.610009,42.300571,24.284752,9981.264932,1.382997,1.112799,30.202904,20.645407,181.066207,456.098091,...,125.338794,66.256028,61.119149,29.317331,55.757415,40.177307,496.123024,2.703626,1.328095,79442.502883
min,1.000000,20.000000,21.000000,1300.000000,1.000000,1.000000,1872.000000,1950.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,2006.000000,34900.000000
25%,365.750000,20.000000,59.000000,7553.500000,5.000000,5.000000,1954.000000,1967.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,5.000000,2007.000000,129975.000000
50%,730.500000,50.000000,69.000000,9478.500000,6.000000,5.000000,1973.000000,1994.000000,0.000000,383.500000,...,0.000000,25.000000,0.000000,0.000000,0.000000,0.000000,0.000000,6.000000,2008.000000,163000.000000
75%,1095.250000,70.000000,80.000000,11601.500000,7.000000,6.000000,2000.000000,2004.000000,166.000000,712.250000,...,168.000000,68.000000,0.000000,0.000000,0.000000,0.000000,0.000000,8.000000,2009.000000,214000.000000
max,1460.000000,190.000000,313.000000,215245.000000,10.000000,9.000000,2010.000000,2010.000000,1600.000000,5644.000000,...,857.000000,547.000000,552.000000,508.000000,480.000000,738.000000,15500.000000,12.000000,2010.000000,755000.000000


## 1) Data Cleaning
- Visualizing data using Plotly.
- Dealing with missing values.

In [11]:
total = df_train.isnull().sum().sort_values(ascending=False)
missing_cols = list(total.index)
total_values = list(total[:])
df_missing = pd.DataFrame(dict({'columns':missing_cols,'total':total_values}))
df_missing = df_missing[df_missing['total']>0]
df_missing

,columns,total
0,PoolQC,1453
1,MiscFeature,1406
2,Alley,1369
3,Fence,1179
4,FireplaceQu,690
5,LotFrontage,259
6,GarageCond,81
7,GarageType,81
8,GarageYrBlt,81
9,GarageFinish,81


In [12]:
fig = px.bar(df_missing, x='columns',y='total')
fig.update_traces(marker_color=dark_blue)
iplot(fig)

In [13]:
missing_data_cols = list(df_missing['columns'])
df_train[missing_data_cols].head()

,PoolQC,MiscFeature,Alley,Fence,FireplaceQu,LotFrontage,GarageCond,GarageType,GarageYrBlt,GarageFinish,GarageQual,BsmtExposure,BsmtFinType2,BsmtFinType1,BsmtCond,BsmtQual,MasVnrArea,MasVnrType,Electrical
0,NaN,NaN,NaN,NaN,NaN,65.0,TA,Attchd,2003.0,RFn,TA,No,Unf,GLQ,TA,Gd,196.0,BrkFace,SBrkr
1,NaN,NaN,NaN,NaN,TA,80.0,TA,Attchd,1976.0,RFn,TA,Gd,Unf,ALQ,TA,Gd,0.0,None,SBrkr
2,NaN,NaN,NaN,NaN,TA,68.0,TA,Attchd,2001.0,RFn,TA,Mn,Unf,GLQ,TA,Gd,162.0,BrkFace,SBrkr
3,NaN,NaN,NaN,NaN,Gd,60.0,TA,Detchd,1998.0,Unf,TA,No,Unf,ALQ,Gd,TA,0.0,None,SBrkr
4,NaN,NaN,NaN,NaN,TA,84.0,TA,Attchd,2000.0,RFn,TA,Av,Unf,GLQ,TA,Gd,350.0,BrkFace,SBrkr


In [14]:
df_train[df_train['PoolArea']>0].shape[0]

7

In [15]:
df_train[df_train['Fireplaces']>0].shape[0]

770

### **Observations: -**

1) PoolQC is a variable which represents Pool Quality. There are not many houses with pools. Hence, most of the values in the PoolArea column (Pool area in sq. ft.) are equal to zero.

2) MiscFeature is used to represent Miscellaneous features. There are not many miscellaneous features.

3) Alley is used to represent the type of alley access to the property. There is alley acces to very less number of properties.

4) Fence is used to represent Fence Quality. Most of the houses do not have a fence.

5) FireplaceQu is used to represent the Quality of the fireplace. Nearly half of the houses in the dataset do not have a fireplace. Hence, some values in the Fireplaces column (No. of fireplaces) are equal to zero.

6) LotFrontage is a column used to denote the linear feet of street connected to the property. Since it contains more than 200 missing values, it is better to remove the column than replacing the missing values with the measures of central tendancy.

### Inferences
- The columns PoolQC, MiscFeature, Alley, Fence and FireplaceQu contain null values not because of errors during data entry, but because of them actually being null.

- The numerical columns PoolArea and Fireplace contain zero values at the null values of PoolQC and FireplaceQu respectively, since it makes sense that the qualitative variables contain null values if the PoolArea is zero

- The same can be said about Fireplaces and FireplaceQu.

In [16]:
cols_to_be_del = ['PoolQC','PoolArea','MiscFeature','Alley','Fence','Fireplaces','FireplaceQu','LotFrontage']
df_train.drop(cols_to_be_del, inplace=True, axis=1)
df_train.shape

(1460, 73)

In [17]:
df_test.head()

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,...,ScreenPorch,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition
0,1461,20,RH,80.0,11622,Pave,NaN,Reg,Lvl,AllPub,...,120,0,NaN,MnPrv,NaN,0,6,2010,WD,Normal
1,1462,20,RL,81.0,14267,Pave,NaN,IR1,Lvl,AllPub,...,0,0,NaN,NaN,Gar2,12500,6,2010,WD,Normal
2,1463,60,RL,74.0,13830,Pave,NaN,IR1,Lvl,AllPub,...,0,0,NaN,MnPrv,NaN,0,3,2010,WD,Normal
3,1464,60,RL,78.0,9978,Pave,NaN,IR1,Lvl,AllPub,...,0,0,NaN,NaN,NaN,0,6,2010,WD,Normal
4,1465,120,RL,43.0,5005,Pave,NaN,IR1,HLS,AllPub,...,144,0,NaN,NaN,NaN,0,1,2010,WD,Normal


In [18]:
compare_df = df_train[df_train['GarageCars']>0]['Id'] == df_train[df_train['GarageArea']>0]['Id']
compare_df.shape[0]

1379

### Inferences:
- There are no abnormal values for Garage variables, i.e., there aren't cases where GarageCars > 0 and GarageArea = 0
- The converse holds true as well, i.e., there aren't cases where GarageArea > 0 and GarageCars = 0
- This means that whoever owns a house with a garage is maintaining atleast one car. Let's keep this in mind.

In [19]:
df_missing.tail(13)

,columns,total
6,GarageCond,81
7,GarageType,81
8,GarageYrBlt,81
9,GarageFinish,81
10,GarageQual,81
11,BsmtExposure,38
12,BsmtFinType2,38
13,BsmtFinType1,37
14,BsmtCond,37
15,BsmtQual,37


### **Observations:-**
7) Although GarageCond, GarageQual are numerical in nature, they are actually categorical.

8) GarageYrBlt, GarageFinish, GarageType are categorical variables which contain null values. These null values cannot be imputed since the null in this context means that there is no garage.

9) There is no record where the GarageCars is zero for non null values of GarageType. This means that every house containing a garage also has a car associated with it.

10) The features BsmtQual, BsmtCond, BsmtCond, BsmtExposure, BsmtFinType1, BsmtFinType2 also follow the same trend.

### Inferences
- The garage variables i.e., the GarageCond, GarageQual, GarageYrBlt, GarageFinish, GarageType can be removed.
- The variables GarageArea, GarageCars however can be useful, since there are only 81 zero values.
- In the same way, BsmtArea is useful to represent the missing values from the corresponding Bsmt variables.

**Note : All of these inferences are made keeping simplicity in mind. Instead of just removing them, one could argue that a careful study of the data can be done to fill in the missing values accordingly. The assumption here is that most of the categorical variables do not affect the target variable heavily since they contain a lot of missing values and they are already being represented in this dataset by their respective categorical values.**

In [20]:
cols_to_be_del_2 = ['GarageCond', 'GarageQual', 'GarageYrBlt', 'GarageFinish', 'GarageType','MasVnrType']
df_train.drop(cols_to_be_del_2, inplace=True, axis=1)
df_train.shape

(1460, 67)

In [21]:
cols_to_be_del_3 = ['BsmtQual','BsmtCond','BsmtCond','BsmtExposure','BsmtFinType1','BsmtFinType2','Electrical']
df_train.drop(cols_to_be_del_3, axis=1, inplace=True)
df_train.shape

(1460, 61)

In [22]:
df_missing.tail(3)

,columns,total
16,MasVnrArea,8
17,MasVnrType,8
18,Electrical,1


In [23]:
df_train['MasVnrArea'].describe()

count    1452.000000
mean      103.685262
std       181.066207
min         0.000000
25%         0.000000
50%         0.000000
75%       166.000000
max      1600.000000
Name: MasVnrArea, dtype: float64

### Inferences
- MasVnrArea has very less number of missing values. Hence, it can be imputed with mean, median or mode.
- To keep things even, let's replace with median since 0 is the most repeated value.

In [24]:
df_train.fillna(df_train.median(), inplace=True)
df_train.isnull().values.sum()

0

## 2) Univariate Analysis of SalePrice
- The ultimtate goal of our analysis is to predict the value of SalePrice with the help of our features.
- Data transformation for skewed variables needs to be done.
- Let's explore the target variable first.

In [25]:
print("Skewness: {}".format(str(df_train['SalePrice'].skew())))
print("Kurtosis: {}".format(str(df_train['SalePrice'].kurt())))

Skewness: 1.8828757597682129
Kurtosis: 6.536281860064529


In [26]:
fig = px.histogram(df_train, x='SalePrice', nbins=100)
iplot(fig)

In [27]:
fig = ff.create_distplot([df_train['SalePrice']],['SalePrice'],bin_size=20000, colors=[dark_blue],show_hist=False)
iplot(fig, filename='Basic Distplot')

In [28]:
def box_plot(dataframe, columns):
    data = []
    for column in columns:
        data.append(go.Box(y=df_train[column], name=column, boxmean='sd',fillcolor=material_blue,marker=dict(color=dark_blue)))
    return data

In [29]:
target_box_data = box_plot(df_train,['SalePrice'])
iplot(target_box_data)

In [30]:
def violin_plot(dataframe, columns):
    data = []
    for column in columns:
        data.append(go.Violin(y=dataframe[column], box_visible=True, line_color=bg_color,
                               meanline_visible=True, fillcolor=material_green, opacity=0.8,
                               x0=column))
    return data

In [31]:
violin_fig = violin_plot(df_train, ['SalePrice'])
iplot(violin_fig, filename = 'SalePriceViolin')

### Observations: -
1) SalePrice is fairly positively skewed, i.e., it is more inclined towards left in the distplot and violin plot.

2) The values above 340k are treated as outliers.

### Inferences: - 
- SalePrice needs to be transformed to a known distribution (preferrably the one which brings the value of skewnwss to zero)

In [32]:
def qqplots(df, col_name, distribution):
    qq = stats.probplot(df, dist=distribution, sparams=(0.5))
    x = np.array([qq[0][0][0],qq[0][0][-1]])
    pts = go.Scatter(x=qq[0][0],
                     y=qq[0][1], 
                     mode = 'markers',
                     showlegend=False,
                     name=col_name,
                     marker = dict(
                            size = 5,
                            color = material_indigo,
                        )
                    )
    line = go.Scatter(x=x,
                      y=qq[1][1] + qq[1][0]*x,
                      showlegend=False,
                      mode='lines',
                      name=distribution,
                      marker = dict(
                            size = 5,
                            color = material_red,
                        )
                     )
    
    data = [pts, line]
    return data

In [33]:
#Plot data for 4 different distributions
norm_data = qqplots(df_train['SalePrice'], 'SalePrice','norm')
power_law_data = qqplots(df_train['SalePrice'], 'SalePrice','powerlaw')
poisson_data = qqplots(df_train['SalePrice'], 'SalePrice','poisson')
lognorm_data = qqplots(df_train['SalePrice'], 'SalePrice','lognorm')

In [34]:
fig = subplots.make_subplots(rows=2, cols=2, subplot_titles=('Normal Distribution', 'Power Law Distribution',
                                                          'Poisson Distribution', 'Log Normal Distribution'))
fig.append_trace(norm_data[0], 1, 1)
fig.append_trace(power_law_data[0], 1, 2)
fig.append_trace(poisson_data[0], 2, 1)
fig.append_trace(lognorm_data[0], 2, 2)
fig['layout'].update(height=600, width=900, title='Comparision of QQ-plots')

iplot(fig, filename='make-subplots-multiple-with-titles')

### Observations: -
3) The Straight line is seen in QQ-plot of SalePrice vs Log-Normal i.e., assuming that the points at the end are outliers.

Let's draw a concrete straight line just to be sure.

In [35]:
layout = dict(xaxis = dict(zeroline = False,
                           linewidth = 1,
                           mirror = True),
              yaxis = dict(zeroline = False, 
                           linewidth = 1,
                           mirror = True),
             )

fig = dict(data=lognorm_data, layout=layout)
iplot(fig, show_link=False)

In [36]:
# Creating a pipeline
df_pipe = df_train.copy()

In [37]:
df_pipe['SalePrice'] = np.log(df_train['SalePrice'])
print("Skewness: {}".format(str(df_pipe['SalePrice'].skew())))
print("Kurtosis: {}".format(str(df_pipe['SalePrice'].kurt())))

Skewness: 0.12133506220520406
Kurtosis: 0.8095319958036296


In [38]:
fig = px.histogram(df_pipe,'SalePrice')
iplot(fig)

In [39]:
fig = ff.create_distplot([df_pipe['SalePrice']],['SalePrice Log Normal'],bin_size=0.08, colors=[dark_blue], show_hist=False)
iplot(fig, filename='Distribution plot for Sale Price (Log transform)')

In [40]:
log_transformed_qqplot_data = qqplots(df_pipe['SalePrice'], 'SalePrice Log transform','norm')
layout = dict(xaxis = dict(zeroline = False,
                       linewidth = 1,
                       mirror = True),
          yaxis = dict(zeroline = False, 
                       linewidth = 1,
                       mirror = True),
         )
qqplot_fig = dict(data=log_transformed_qqplot_data, layout=layout)
iplot(qqplot_fig, show_link=False)

In [41]:
target_transformed_box_data = [go.Box(y=df_pipe['SalePrice'], name='SalePrice Log transform', boxmean='sd',fillcolor=material_green,marker=dict(color=dark_green))]
iplot(target_transformed_box_data)

In [42]:
target_transformed_violin_data = violin_plot(df_pipe,['SalePrice'])
iplot(target_transformed_violin_data, filename = 'SalePriceLogViolin', validate = False)

In [43]:
fig = subplots.make_subplots(rows=1, cols=2)
fig.append_trace(target_box_data[0], 1, 1)
fig.append_trace(target_transformed_box_data[0], 1, 2)
fig['layout'].update(height=600, width=950, title='SalePrice Unchanged vs Log transformed')
iplot(fig, filename='SalePrice-unch-vs-log-box')

In [44]:
print("Skewness: {}".format(str(df_pipe['SalePrice'].skew())))
print("Kurtosis: {}".format(str(df_pipe['SalePrice'].kurt())))

Skewness: 0.12133506220520406
Kurtosis: 0.8095319958036296


#### Transformation Success!
- Skewness is down significantly,
- SalePrice is now closer to a normal distribution.

## 3) EDA and Feature Engineering
- Combination of certain features.
- Creating age based on year given.
- Visualization of numerical features.

In [45]:
#Feature Engineering

df_pipe['TotalSF']=df_pipe['TotalBsmtSF'] + df_pipe['1stFlrSF'] + df_pipe['2ndFlrSF']
df_pipe['TotalSQR_Footage'] = (df_pipe['BsmtFinSF1'] + df_pipe['BsmtFinSF2'] +
                                df_pipe['1stFlrSF'] + df_pipe['2ndFlrSF'])

df_pipe['Total_Bathrooms'] = (df_pipe['FullBath'] + (0.5 * df_pipe['HalfBath']) +
                              df_pipe['BsmtFullBath'] + (0.5 * df_pipe['BsmtHalfBath']))

df_pipe['AgeSinceRemodel'] = 2010 - df_train['YearRemodAdd']
df_pipe['AgeSinceBuilt'] = 2010 - df_train['YearBuilt']

In [46]:
corr_matrix = df_pipe.corr()
corr_matrix = corr_matrix.abs()
fig = go.Figure(data=go.Heatmap(z=corr_matrix))
iplot(fig)

In [47]:
k = 15 #number of variables for heatmap
cols = corr_matrix.nlargest(k, 'SalePrice')['SalePrice'].index

cols_to_be_del_pipe = ['FullBath','1stFlrSF']
cols = list(cols)
for i in cols_to_be_del_pipe:
    cols.remove(i)

correlations = list(df_pipe[cols].corr().round(2).values)
correlation_matrix = [list(correlations[i]) for i in range(len(correlations))]
fig = ff.create_annotated_heatmap(z=correlation_matrix,x=list(cols),y=list(cols))
iplot(fig, filename='annotated_heatmap')

In [48]:
#Categorical variables
ordinal_cols = ['BldgType','HeatingQC','Functional']
binary_cols = ['PavedDrive','CentralAir']
df_pipe[ordinal_cols] = df_pipe[ordinal_cols].astype('category')
df_pipe[ordinal_cols].head()

,BldgType,HeatingQC,Functional
0,1Fam,Ex,Typ
1,1Fam,Ex,Typ
2,1Fam,Ex,Typ
3,1Fam,Gd,Typ
4,1Fam,Ex,Typ


In [49]:
def count_plot(df,col_name):
    value_counts_series = df[col_name].value_counts()
    categories = list(value_counts_series.index)
    values = list(value_counts_series)
    fig = go.Figure(data=[go.Bar(
            x=categories, 
            y=values,
            textposition='auto',
        )])
    iplot(fig)

In [50]:
count_plot(df_pipe, 'HeatingQC')

In [51]:
fig = px.box(df_train, x='HeatingQC', y='SalePrice')
iplot(fig)

In [52]:
# Label Encoding for HeatingQC
categories = list(df_pipe['HeatingQC'].unique())
encoding_dict = {col:x for col,x in zip(categories,range(5,0,-1))}
replace_dict_heatingqc = {'HeatingQC':encoding_dict}
df_pipe.replace(replace_dict_heatingqc, inplace = True)
df_pipe[ordinal_cols].head()

,BldgType,HeatingQC,Functional
0,1Fam,5,Typ
1,1Fam,5,Typ
2,1Fam,5,Typ
3,1Fam,4,Typ
4,1Fam,5,Typ


In [53]:
skews = []
kurts = []
for col in cols:
    skews.append(df_pipe[col].skew())
    kurts.append(df_pipe[col].kurt())
dict_skew_data = {'Feature':cols, 'Skew':skews, 'Kurt':kurts}
df_skews = pd.DataFrame(dict_skew_data, columns=['Feature','Skew','Kurt'])
df_skews

,Feature,Skew,Kurt
0,SalePrice,0.121335,0.809532
1,OverallQual,0.216944,0.096293
2,TotalSF,1.776700,12.621968
3,GrLivArea,1.366560,4.895121
4,TotalSQR_Footage,2.163968,16.150922
5,GarageCars,-0.342549,0.220998
6,Total_Bathrooms,0.264676,-0.133701
7,GarageArea,0.179981,0.917067
8,TotalBsmtSF,1.524255,13.250483
9,YearBuilt,-0.613461,-0.439552


In [54]:
df_pipe['TotalSF'] = np.log(df_pipe['TotalSF'])

In [55]:
iplot(qqplots(df_pipe['TotalSF'],'TotalSF Log transform','norm'))

In [56]:
fig = px.scatter_matrix(df_pipe, dimensions=['TotalSF','GrLivArea','TotalSQR_Footage','SalePrice'],color='OverallQual')
iplot(fig)

In [57]:
fig = px.scatter_matrix(df_pipe, dimensions=['Total_Bathrooms','GarageArea','GarageCars','SalePrice'],color='OverallQual')
iplot(fig)

In [58]:
fig = px.scatter_matrix(df_pipe, dimensions=['AgeSinceBuilt','AgeSinceRemodel','TotRmsAbvGrd','TotalBsmtSF','SalePrice'],color='OverallQual')
iplot(fig)

In [59]:
def draw_scatter_plot(col_name_x, col_name_y):
    trace = go.Scatter(
        x = df_pipe[col_name_x],
        y = df_pipe[col_name_y],
        mode = 'markers'
    )
    data = [trace]
    iplot(data, filename='basic-scatter')

## 4) LASSO (Linear regression)
- The model which can deal with collinearity due to L1 regualizer (Since it takes the absolute value, it can penalize unnecessary variables and make them equal to zero).
- One of the most basic models.

In [60]:
y_tr = df_pipe['SalePrice']
x_tr = df_pipe[cols]
lasso = linear_model.Lasso()
parameters = {'alpha': [1]}
lasso_regressor = GridSearchCV(lasso, parameters, scoring=make_scorer(metrics.mean_squared_error), cv=10)
lasso_regressor.fit(x_tr, y_tr)
y_pred_lasso = lasso_regressor.predict(x_tr)

In [61]:
mse = metrics.mean_squared_error(y_tr, y_pred_lasso, sample_weight=None)
rmse = np.sqrt(mse)
print(rmse)

0.1953275255822847


## Conclusions:
1) So far, we have explored the numerical variables to a certain extent. However, most of the categorical variables remain to be explored.

2) The feature selection of the numerical variables was based on correlations and scatterplot matrices. This should suffice for a basic approach.

3) The Kernel will be continued . If you are readng this, thank you for checking out this kernel. It must've been tough, since I've included a lot of visualizations and I could not document some of my decisions.

4) Please leave a message in the discussions if there's more to be included.

### Thank you!

### References:

**1) Plotly**
- https://plot.ly/python/
- https://www.kaggle.com/thebrownviking20/intermediate-visualization-tutorial-using-plotly
- https://www.kaggle.com/artemseleznev/plotly-tutorial-for-beginners
- https://stackoverflow.com/questions/51170553/qq-plot-using-plotly-in-python

**2) Kernels**
- https://www.kaggle.com/pmarcelino/comprehensive-data-exploration-with-python
- https://www.kaggle.com/hamzaben/eda-feature-eng-and-model-blending-top-20
- https://www.kaggle.com/erikbruin/house-prices-lasso-xgboost-and-a-detailed-eda
- https://www.kaggle.com/masumrumi/a-stats-analysis-and-ml-workflow-of-house-pricing
- https://www.kaggle.com/mjbahmani/the-data-scientist-s-toolbox-tutorial-1